In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [2]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

from surprise import accuracy, Dataset, SVD,SVDpp, NMF 
from surprise.model_selection import train_test_split as train_test_split_surprise
from surprise.model_selection import cross_validate, GridSearchCV
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import numpy as np


In [3]:
TRAIN__Validation_SIZE = 0.8
TEST_SIZE = 0.2

### Read the Train and Test

In [4]:
train_validation = pd.read_csv("../../data/recommendation/ad_interaction_train.csv")
test_set = pd.read_csv("../../data/recommendation/ad_interaction_test.csv")
train_validation.shape,test_set.shape

((905893, 6), (226474, 6))

### Cross validate on multiple Model types

In [5]:
reader = Reader(rating_scale=(1, 5))
cv_data = Dataset.load_from_df(train_validation[["user_id",\
                                                 "recipe_id", "rating"]], reader)

In [6]:
param_grid = {"n_factors":[5, 25] ,"n_epochs": [20, 250], "lr_all": [0.001, 0.006],\
             "reg_all":[0.01,0.08]}
param_grid2 = {"n_factors":[5, 25] ,"n_epochs": [2, 20], "reg_pu":[0.01,0.1], "reg_qi":[0.01,0.1]}

#### SVD, SVDpp, NMF

In [7]:
gs_svd = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -2, joblib_verbose=3)
# gs_svdpp = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -2, joblib_verbose=3)
gs_nmf = GridSearchCV(NMF, param_grid2, measures=["rmse", "mae"], cv=5, n_jobs = -2, joblib_verbose=3)

In [8]:
gs_nmf.fit(cv_data)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:  3.2min finished


In [9]:
gs_svd.fit(cv_data)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:  7.8min finished


In [10]:
# gs_svdpp.fit(cv_data)

#### Compare results

In [17]:
gs_nmf.best_score["rmse"]

1.3048597107504085

In [18]:
gs_result = {}
for algo,gs in {"svd":gs_svd,"nmf":gs_nmf}.items():
    gs_result[algo] = [gs.best_score["rmse"],gs.best_score["mae"]]

In [19]:
gs_result_df = pd.DataFrame.from_dict(gs_result)
gs_result_df.index = ["rmse","mae"]
gs_result_df

,svd,nmf
rmse,1.217126,1.304860
mae,0.737252,0.653033


In [20]:
gs_svd.best_params["rmse"]

{'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.006, 'reg_all': 0.08}

In [34]:
gs_nmf.best_params["mae"]

{'n_factors': 25, 'n_epochs': 2, 'reg_pu': 0.01, 'reg_qi': 0.01}

#### Build model on full cv set

In [44]:
model1 = gs_svd.best_estimator["rmse"]
model1.fit(cv_data.build_full_trainset())

model2 = gs_nmf.best_estimator["mae"]
model2.fit(cv_data.build_full_trainset())

#### Write models to pickle

In [45]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

In [46]:
from src.utilities import *

write_pickle(model1, "../../models/reccomender_model1_svd.pkl")
write_pickle(model2, "../../models/reccomender_model2_nmf.pkl")

Pickle Dump Falied
Pickle Dump Falied


In [47]:
test_set_surprise = Dataset.load_from_df(test_set[["user_id", "recipe_id", "rating"]], reader)
_, test_set_surprise = train_test_split_surprise(test_set_surprise, test_size=1.0)

In [50]:
predictions1 = model1.test(test_set_surprise)
accuracy.rmse(predictions1, verbose=False), accuracy.mae(predictions1, verbose=False)

(1.2106318289575184, 0.7378288018266064)

In [51]:
predictions2 = model2.test(test_set_surprise)
accuracy.rmse(predictions2, verbose=False), accuracy.mae(predictions2, verbose=False)

(1.3171728600922699, 0.64265160294064)